# Collect Weather Station data around chosen polygon

In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

from dataclasses import dataclass

import arrow

import contextily as ctx
import datetime

import requests

@dataclass
class Station:
    name: str
    lat: float
    lon: float
    temperature: list
    wind_speed: list
    wind_direction: list
    timestamp: list
    units: dict

/opt/conda/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_319/892420718.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling P

In [5]:
def reproject_geom(geom, from_crs='EPSG:5070', to_crs='EPSG:3857'):
    return gpd.GeoSeries(geom, crs=from_crs).to_crs(to_crs)[0]


def get_observation(description : str, tix : int, crs_epsg=5070):
    ''' Obtain selected observation geometry and datetime
    '''

    if description not in ['Maria2019', 'River2021', 'Bridge2021']:
        raise ValueError(f'description {description} not present in db')
    
    def change_username_jovyan(df, column):
        for ix, row in df.iterrows():
            path_list = row[column].split('/')
            path_list[2] = 'jovyan'
    
            path = ''
            for string in path_list[:-1]:
                path += f'{string}/'
            path += path_list[-1]
    
            df.loc[ix, column] = path  
    
    df = pd.read_pickle('/home/jovyan/data/dftable_06032023.pkl')
    change_username_jovyan(df, 'filepath')
    
    dfrow = df[df['description'] == description].sort_values('datetime').iloc[tix]
    dfgeom = gpd.read_file(dfrow['filepath']).to_crs(epsg=crs_epsg)['geometry'][0]
    dfdt = dfrow['datetime']
    
    return dfgeom, dfdt


# def collect_wx_data(gdf, gdfignition, igniteidx, compareidx):
def collect_wx_data(centerlon, centerlat, radius, fromdt, todt):
    km_per_miles = 1.60934

    # ignite = gdfignition.to_crs(epsg=4326).loc[igniteidx]
    # compare = gdfignition.to_crs(epsg=4326).loc[compareidx]

    baseURL = "https://firemap.sdsc.edu/pylaski/stations/data?"

    ## Always start with the base of the URL

    selectionType='selection=withinRadius'

    # Choose the centroid of the ignition
    # lon = ignite['geometry'].centroid.x
    # lat = ignite['geometry'].centroid.y
    lon = centerlon
    lat = centerlat
    
    # radius = 5*km_per_miles    # 2 mile radius

    ## Latitude and longitude of the White House according to Google
    selectionParameters = '&lat={}&lon={}&radius={}'.format(str(lat),str(lon), str(radius))

    observables_list = ['temperature', 'wind_speed', 'wind_direction']
    observables = ''
    for observable in observables_list:
        observables += f'&observable={observable}'

    # fromdt = ignite['datetime'].tz_localize('US/Pacific')
    # todt = compare['datetime'].tz_localize('US/Pacific')
    frm = arrow.Arrow.fromdatetime(fromdt.to_pydatetime())
    to = arrow.Arrow.fromdatetime(todt.to_pydatetime())

    frm = frm.shift(minutes=-10)
    to = to.shift(minutes=10)

    urlDateTime = '&from={}&to={}'.format(str(frm), str(to))

    urlPlot = baseURL + selectionType + selectionParameters + observables + urlDateTime
    print(urlPlot)

    r = requests.get(urlPlot)
    ## Request to GET information from the given URL (Our REST query we built)
    r_json = r.json()

    stations_lst = []
    for feature in r_json['features']:

        lon,lat = feature['geometry']['coordinates']

        name = feature['properties']['description']['name']
        wxid = feature['properties']['description']['id']
        provider = feature['properties']['description']['provider']
        wifire_uid = feature['properties']['description']['wifire_uid']

        timestamp = pd.to_datetime(feature['properties']['timestamp'])
        try:
            temperature = feature['properties']['temperature']
        except KeyError:
            temperature = []

        try:
            wind_speed = feature['properties']['wind_speed']
        except KeyError:
            wind_speed = []

        try:
            wind_direction = feature['properties']['wind_direction']
        except KeyError:
            wind_direction = []

        units = feature['properties']['units']

        stations_lst.append(Station(name, lat, lon, temperature, wind_speed, wind_direction, timestamp, units))
        
    return stations_lst

In [6]:
maria_0, dt = get_observation('Maria2019', 0, crs_epsg=4326)

In [7]:
maria_0.centroid.xy

(array('d', [-119.03909139010646]), array('d', [34.31984085482562]))

In [8]:
fromdt = dt.tz_localize('US/Pacific')-datetime.timedelta(minutes=10)
todt = dt.tz_localize('US/Pacific')+datetime.timedelta(minutes=10)

year = 2023
fromdt = fromdt.replace(year=year)
todt = todt.replace(year=year)

In [14]:
stations_lst = collect_wx_data(maria_0.centroid.x, maria_0.centroid.y, radius=5, fromdt=fromdt, todt=todt)

https://firemap.sdsc.edu/pylaski/stations/data?selection=withinRadius&lat=34.31984085482562&lon=-119.03909139010646&radius=5&observable=temperature&observable=wind_speed&observable=wind_direction&from=2023-10-31T19:17:11-07:00&to=2023-10-31T19:57:11-07:00


In [22]:
stations_lst[-3:]

[Station(name='SCE Krohn Ridge', lat=34.30253, lon=-119.06011, temperature=[74.89, 75.19, 74.7, 75.8], wind_speed=[1.57, 1.21, 1.62, 1.27], wind_direction=[338.5, 347.2, 0.89, 339.1], timestamp=DatetimeIndex(['2023-10-31 19:20:00-07:00', '2023-10-31 19:30:00-07:00',
                '2023-10-31 19:40:00-07:00', '2023-10-31 19:50:00-07:00'],
               dtype='datetime64[ns, UTC-07:00]', freq=None), units={'temperature': 'F', 'wind_speed': 'mps', 'wind_direction': 'd'}),
 Station(name='SCE Morgan Canyon Rd', lat=34.34512, lon=-119.04721, temperature=[64.49, 63.1, 62.44, 62.4], wind_speed=[1.2, 1.22, 1.42, 0.94], wind_direction=[159.2, 152.8, 162.7, 151.7], timestamp=DatetimeIndex(['2023-10-31 19:20:00-07:00', '2023-10-31 19:30:00-07:00',
                '2023-10-31 19:40:00-07:00', '2023-10-31 19:50:00-07:00'],
               dtype='datetime64[ns, UTC-07:00]', freq=None), units={'temperature': 'F', 'wind_speed': 'mps', 'wind_direction': 'd'}),
 Station(name='SCE River Valley View', la